# Test RC Input Logging from second RX

In [1]:
from flightdata import Flight, Fields, CIDTypes

surfaces = {
    'r_aileron': 'tx_controls_1',
    'l_aileron': 'tx_controls_2',
    'elevator': 'tx_controls_3',
    'rudder': 'tx_controls_4',
}

deflection_bins = {key: Flight.from_log('./rc_inputs/' + key + '.BIN') for key in surfaces.keys() }

FileNotFoundError: [Errno 2] No such file or directory: './rc_inputs/r_aileron.BIN'

In [3]:
from geometry import Point
from geometry.point import cross_product

import numpy as np
import pandas as pd
def get_angle(x1, y1, z1, x2, y2, z2):
    a = Point(x1, y1, z1)
    b = Point(x2, y2, z2)
    sinthetanhat = cross_product(a, b) / (abs(a) * abs(b) )
    return sinthetanhat.y

def make_deflection_df(bin, tx_control):
    angledata = bin.read_fields(Fields.ACCELERATION)
    
    angledata.columns = 'x2,y2,z2'.split(',')
    angledata.insert(0, 'z1', angledata.iloc[0].z2)
    angledata.insert(0, 'y1', angledata.iloc[0].y2)
    angledata.insert(0, 'x1', angledata.iloc[0].x2)

    angledata['angle'] = np.degrees(np.arcsin(np.vectorize(get_angle)(*tuple(angledata.transpose().to_numpy()))))


    angledata['input_pwm'] = bin.data[tx_control]
    return angledata[['angle', 'input_pwm']]

deflection_dfs = {surf: make_deflection_df(deflection_bins[surf], surfaces[surf]) for surf in surfaces.keys()}

deflection_dfs

{'r_aileron':                angle  input_pwm
 time_index                     
 0.000000    0.000000     1569.0
 0.099988   -0.067705     1568.0
 0.199988   -0.022299     1568.0
 0.299987   -0.051594     1568.0
 0.400006   -0.012314     1568.0
 ...              ...        ...
 102.500387 -0.041365     1569.0
 102.599891  0.597682     1570.0
 102.699894 -0.499186     1570.0
 102.799876  1.507838     1569.0
 102.899868 -1.080860     1570.0
 
 [1030 rows x 2 columns],
 'l_aileron':                angle  input_pwm
 time_index                     
 0.000000    0.000000     1490.0
 0.100059    0.074559     1491.0
 0.199999    0.164140     1491.0
 0.299993    0.046475     1490.0
 0.399966    0.134942     1491.0
 ...              ...        ...
 97.299925  -1.399497     1490.0
 97.399920  -0.211978     1490.0
 97.499924   0.579031     1491.0
 97.599923  -0.411621     1491.0
 97.699803   1.306930     1490.0
 
 [978 rows x 2 columns],
 'elevator':                angle  input_pwm
 time_index     

In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

angledata = deflection_dfs['rudder']
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=angledata.index, y=angledata.input_pwm, name="pwm"),secondary_y=False)
fig.add_trace(go.Scatter(x=angledata.index, y=angledata.angle, name="angle"),secondary_y=True)

fig.update_xaxes(title_text="xaxis title")

fig.update_yaxes(title_text="input_pwm", secondary_y=False)
fig.update_yaxes(title_text="angle(deg)", secondary_y=True)

fig.show()

In [13]:
rate_regions = {
    'rudder':{
        'landing': [0,38],
        'line': [38,66],
        'radius1':[38,66],
        'radius2':[66,78],
        'radius3':[78,88],
        'stallturn': [108,126],
        'spin': [88,108],
        'snap': [126,140]
    },
    'elevator':{
        'landing': [0, 10.5],
        'line': [26, 36],
        'radius1': [37, 44],
        'radius2': [44, 48],
        'radius3': [48, 54],
        'stallturn': [48, 54],
        'spin': [0, 10.5],
        'snap': [44, 48]
    },
    'r_aileron':{
        'landing': [12, 33],
        'line': [33,51],
        'radius1':[51,60],
        'radius2':[60,68],
        'radius3':[68,75],
        'stallturn': [75, 83],
        'spin': [83, 90],
        'snap': [92, 98]
    },
    'l_aileron':{
        'landing': [0, 15],
        'line': [15,28],
        'radius1':[28,37],
        'radius2':[37,44],
        'radius3':[44,55],
        'stallturn': [55,67],
        'spin': [67,78],
        'snap': [67,92]
    }
    }



In [47]:


class RateInfo():
    def __init__(self, angledata, regions):
        self.centre = angledata.input_pwm.median()
        self.func = np.polyfit(angledata.input_pwm, angledata.angle,3)
        self.ratedata = {key: angledata.loc[value[0]:value[1]] for key, value in regions.items()}
        self.rates = {key: [value.input_pwm.min(), value.input_pwm.max()] for key, value in self.ratedata.items()}
        self.angles = {key: [self.deflection(value[0]), self.deflection(value[1])] for key, value in self.rates.items()}        

    def deflection(self, pwm):
        return self.func[0] * pwm**3 + self.func[1] * pwm**2 + self.func[2] * pwm + self.func[3]

rateinfo = {}
for key in rate_regions.keys():
    rateinfo[key] = RateInfo(deflection_dfs[key], rate_regions[key])


In [46]:

for key, value in rateinfo.items():
    fig = go.Figure()

    pwm_values = np.linspace(value.rates['landing'][0], value.rates['landing'][1]) 
    deflections = np.vectorize(value.deflection)(pwm_values)


    line = fig.add_trace(go.Scatter(x=pwm_values, y=deflections, name='full range'))

    for rate in value.rates.keys():
        if rate in 'landing,line,radius1'.split(','):
            fig.add_trace(go.Scatter(
                x=value.rates[rate], 
                y=value.angles[rate], 
                text=["{:.2f}".format(val) for val in value.angles[rate]], 
                mode='markers+text',
                textposition="bottom right", 
                name=rate
                ))

    fig.update_layout(title= key + ' Deflection vs pwm value')
    fig.update_xaxes(title_text="signal pulse width")

    fig.update_yaxes(title_text="deflection (degrees)")
    fig.show()